# Assignment 1: Fine-tuning for text classification of Japanese review

- Choose the appropriate pre-trained model for fine-tuning.
- Examine the performance of the fine-tuned model on the test dataset.
- Use the fine-tuned model to infer the sentiment of raw text.

```
text1="使い物にならない"
text2="とても気に入った"
```

In [1]:
%pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

In [2]:
!nvidia-smi

Thu Jun 12 06:02:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install transformers
!pip install datasets

In [4]:
from datasets import load_dataset
from datasets import load_dataset, DatasetDict

dataset = load_dataset(
    "tyqiangz/multilingual-sentiments",
    "japanese")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

multilingual-sentiments.py:   0%|          | 0.00/6.23k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/507k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/517k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [5]:
# Shuffle the dataset
dataset = dataset.shuffle(seed=42)

# Subset the dataset
train_dataset = dataset['train'].select(range(3000))
valid_dataset = dataset['train'].select(range(3000, 3800))
test_dataset = dataset['train'].select(range(3800, 4600))

# Compile them into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 800
    })
})

In [7]:
dataset.set_format(type="pandas")
train_df = dataset["train"][:]
train_df.head(5)

,text,source,label
0,4つ購入したが、2つは使用できず。接触が悪いのかわからないが、2度と購入しません,amazon_reviews_multi,2
1,キャンプでカセットコンロを使用しています。 キャンプでちょっと変わった事がしたくて購入！笑 ...,amazon_reviews_multi,0
2,こちらのiTunesカード200＄分を購入しました。 50＄４枚購入したはずが、届いたのは5...,amazon_reviews_multi,2
3,macbook pro 13インチと書いてあったので書いてあったので購入しましたが2016l...,amazon_reviews_multi,1
4,安かろう悪かろうです。 TVはが悪く暗く自動サーチなしでいちいちサーチしなおさなければダメ。...,amazon_reviews_multi,2


In [8]:
dataset.reset_format()

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cl-tohoku/bert-base-japanese"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [38]:
import numpy as np
import evaluate

# 加载 accuracy 指标
accuracy_metric = evaluate.load("accuracy")

# 定义评估函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [39]:
#设置参数和trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

In [20]:
# 训练模型
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.901098
2,0.319000,0.897186
3,0.240000,1.091774


TrainOutput(global_step=1125, training_loss=0.2666169620090061, metrics={'train_runtime': 940.8764, 'train_samples_per_second': 9.566, 'train_steps_per_second': 1.196, 'total_flos': 2368020759552000.0, 'train_loss': 0.2666169620090061, 'epoch': 3.0})

In [40]:
#评估
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 1.1362768411636353,
 'eval_model_preparation_time': 0.0109,
 'eval_accuracy': 0.755,
 'eval_runtime': 23.2501,
 'eval_samples_per_second': 34.409,
 'eval_steps_per_second': 4.301}

In [44]:
text1 = "使い物にならない"
text2 = "とても気に入った"

inputs = tokenizer([text1, text2], return_tensors="pt", padding=True, truncation=True)

device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

label_names = dataset["train"].features["label"].names
label_map = {i: name.capitalize() for i, name in enumerate(label_names)}

for text, label in zip([text1, text2], predictions):
    print(f"「{text}」 → 感情: {label_map[label.item()]}")



「使い物にならない」 → 感情: Negative
「とても気に入った」 → 感情: Positive


In [10]:
!pip install fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 85.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.4/698.4 kB 27.5 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=4d65fa2eafcf1b95e0c864e728b1d659dce5d4c80a7cc223dfb8ea2c1a8fab45
  Stored in directory: /root/.cache/pip/wheels/44/56/37/f543963822b85260c9f948df8fac8c20169c80dc71b24dc407
Successfully built ipadic


In [11]:
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=cc3c14c30bfba2779ad2a40bab7595a96c1f2eadfde0fa0d1c421b6ce772d653
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built unidic-lite


In [37]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.6 MB/s eta 0:00:00
